# Testing generating RH response plots

In [ ]:
import datetime
import pandas as pd
import pyodbc
import json

#import plotly
import plotly.graph_objects as go

In [ ]:
#SQL Server connection info
with open("./../.dbCreds") as f:
	dbCreds = json.load(f)

# Formatted connection string for the SQL DB.
SQL_CONN_STR = 'DSN=Salford-SQL-Server;Database=salfordMOVE;Trusted_Connection=no;UID='+dbCreds['UNAME']+';PWD='+dbCreds['PWD']+';'

In [ ]:
conn = pyodbc.connect(SQL_CONN_STR)

SQL = """
      SELECT r.[messageDate]
            ,r.[plotValue]
      FROM [salfordMove].[dbo].[READINGS] AS r
      JOIN [salfordMOVE].[dbo].SENSORS AS s
            ON (r.sensorID = s.sensorID)
      JOIN [salfordMOVE].[dbo].PLOT_LABELS as pl
            ON (r.plotLabelID = pl.plotLabelID)
      WHERE [sensorName] LIKE 'RH%'
            AND [sensorName] LIKE '%Z03%'
            AND [plotLabel] = 'Humidity'
      ORDER BY messageDate ASC
"""

result = pd.read_sql(SQL, conn)

In [ ]:
# Round the timestamps to the closest 15min
#result['messageDate'] = result['messageDate'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*round((float(dt.minute) + float(dt.second)/60) / 15)))
result['messageDate']=result['messageDate'].dt.round('15min')

In [ ]:
# Set the upper and lower limits for the RH chart
upper_limits = [4, 13, 19, 30, 39, 49, 60, 74, 78, 87, 100]
lower_limits = [0, 7, 12, 20, 27, 36, 47, 63, 71, 83, 100]

# Set the response time & n steps for calculating RH response
# 0.416667, 1, 4.3, 15, 26, 30, 40
response_time = {"RHi10h" : 40, "RHi1d" : 96, "RHi4.3d" : 412.8, "RHi15d" : 1440, "RHi26d" : 2496, "RHi30d" : 2880, "RHi40d" : 3840}

# Load test data from file
#data = pd.read_csv('dataSampleVisualisations.csv')
#data_10h = data['10h']
#data_15d = data['15d']

In [ ]:
# calculate the RH respsonse datasets here
# RHresponse,i = RHresponse,i - 1 / (RHi / (n/3)) / 1 + (1 / (n / 3))
## RHresponse,i = RH at the current time step
## RHresponse,i - 1 = RH at the previous time step
## RHi = RH of the air at time step
## n = number of time steps for the tiem span

data_10h = pd.DataFrame(result.head(1))
data_15d = pd.DataFrame(result.head(1))

for i in range(len(result.index)):
    if i != len(result.index) - 1:
        o = i + 1
    else:
        o = i

    v_10h = (float(data_10h.iloc[i, 1]) + (float(result.iloc[o, 1]) / (response_time["RHi10h"] / 3))) / (1 + (1 / (response_time["RHi10h"] / 3)))
    v_15d = (float(data_15d.iloc[i, 1]) + (float(result.iloc[o, 1]) / (response_time["RHi15d"] / 3))) / (1 + (1 / (response_time["RHi15d"] / 3)))

    # Compile the current timestamp and calculated value into a dict
    data_10h_temp = {"messageDate" : result.iloc[o, 0], "plotValue" : v_10h}
    data_15d_temp = {"messageDate" : result.iloc[o, 0], "plotValue" : v_15d}

    # Append the newly calculated value to the DF
    data_10h = data_10h.append(data_10h_temp, ignore_index=True)
    data_15d = data_15d.append(data_15d_temp, ignore_index=True)

    print("\r" + str(i), end="")


In [ ]:
#print(data_10h)
#print(data_15d)

In [ ]:
# Instantiate the figure object
fig = go.Figure()

# Add traces to the figure
fig.add_trace(go.Scatter(x = lower_limits, y = upper_limits, mode = 'lines', name = 'Upper Limit', marker_color = 'Red'))
fig.add_trace(go.Scatter(x = data_10h["plotValue"], y = data_15d["plotValue"], mode = 'lines+markers', name = 'Data', marker_color = 'Blue'))
fig.add_trace(go.Scatter(x = upper_limits, y = lower_limits, mode = 'lines', name = 'Lower Limit', marker_color = 'Orange'))
fig.update_layout(title="RH Response - Zone 03", xaxis_title="10h", yaxis_title="15d", width=1000, height=1000)